In [1]:
import os
import zipfile
import shutil

# Chemin du fichier ZIP
zip_path = r"C:\Users\Tanoh Parfait\Downloads\extraction_du_09-11-2025_au_11-11-2025.zip"

# Répertoire temporaire d’extraction
extract_dir = r"D:\RECONCILIATIONS\TEMP"

# Mapping du dossiers ZIP → dossiers existants
MAPPING = {
    ("Payment", "CINETPAY"): r"D:\RECONCILIATIONS\PAIEMENT\Cinetpay\DONNEES PMT\2025\NOV",
    ("Transfer", "CINETPAY"): r"D:\RECONCILIATIONS\TRANSFERT\Cinetpay\DONNEES PMT\2025\NOV",
    ("Transfer", "MTN_CAMEROUN"): r"D:\RECONCILIATIONS\TRANSFERT\MTN CM\Données PMT\2025\NOV",
    ("Payment", "MTN_CAMEROUN"): r"D:\RECONCILIATIONS\PAIEMENT\MTN CM\donnees PMT\2025\NOV",
    ("Payment", "ORANGE_BURKINA"): r"D:\RECONCILIATIONS\PAIEMENT\OM BF\Données PMT\2025\NOV",
    ("Payment", "ORANGE_SENEGAL"): r"D:\RECONCILIATIONS\PAIEMENT\OM_SENEGAL\Données_PMT\2025\NOV",
    ("Transfer", "ORANGE_SENEGAL"): r"D:\RECONCILIATIONS\TRANSFERT\ORANGE SN\Données PMT\2025\NOV",
    ("Payment", "WAVE_CI"): r"D:\RECONCILIATIONS\PAIEMENT\WAVE-CI\Données PMT\NOV",
    ("Transfer", "WAVE_CI"): r"D:\RECONCILIATIONS\TRANSFERT\WAVE\Données PMT\NOV",
    ("Payment", "WAVE_BURKINA"): r"D:\RECONCILIATIONS\PAIEMENT\WAVE BF\DONNEES PMT\NOV",
    ("Transfer", "WAVE_BURKINA"): r"D:\RECONCILIATIONS\TRANSFERT\WAVE BF\DONNEES PMT\NOV",
    ("Payment", "IFUTUR"): r"D:\RECONCILIATIONS\PAIEMENT\IFUTUR\DONNEES PMT\NOV",
    ("Transfer", "IFUTUR"): r"D:\RECONCILIATIONS\TRANSFERT\IFUTUR\DONNEES PMT\NOV",
    ("Payment", "PAYDUNYA"): r"D:\RECONCILIATIONS\PAIEMENT\PAYDUNYA\Données PMT\NOV",
    ("Transfer", "PAYDUNYA"): r"D:\RECONCILIATIONS\TRANSFERT\PAYDUNYA\Données PMT",
    ("Payment", "MOOVBF"): r"D:\RECONCILIATIONS\PAIEMENT\MOOVBF\DATA PMT\NOV",
    ("Transfer", "MOOVBF"): r"D:\RECONCILIATIONS\TRANSFERT\MOOVBF\DATA PMT\NOV",
    ("Payment", "WIZALL_SENEGAL"): r"D:\RECONCILIATIONS\PAIEMENT\WIZALL\DONNEES PMT",
    ("Transfer", "WIZALL_SENEGAL"): r"D:\RECONCILIATIONS\TRANSFERT\WIZALL\DONNEES PMT",
    ("Payment", "MTN_CI"): r"D:\RECONCILIATIONS\PAIEMENT\MTN CI\donnees PMT\NOV",
    ("Transfer", "MTN_CI"): r"D:\RECONCILIATIONS\TRANSFERT\MTN CI\donnees PMT\NOV",
    ("Payment", "AIRTEL_TCHAD"): r"D:\RECONCILIATIONS\PAIEMENT\AIRTEL TD\DONNEES PMT",
    ("Payment", "MOOV_NE"): r"D:\RECONCILIATIONS\PAIEMENT\MOOV NE\DONNEES PMT\NOV",
    ("Payment", "ORANGE_CD"): r"D:\RECONCILIATIONS\PAIEMENT\ORANGE CD\DONNESS PMT\NOV",
    ("Transfer", "ORANGE_CD"): r"D:\RECONCILIATIONS\TRANSFERT\ORANGE CD\DONNESS PMT\NOV",
    ("Payment", "AIRTEL_RDC"): r"D:\RECONCILIATIONS\PAIEMENT\AIRTEL CD\DONNESS PMT\NOV",
    ("Transfer", "AIRTEL_RDC"): r"D:\RECONCILIATIONS\TRANSFERT\AIRTEL CD\DONNESS PMT\NOV",
    ("Payment", "WAVE_SENEGAL"): r"D:\RECONCILIATIONS\PAIEMENT\WAVE - SN\Données PMT\NOV",
    ("Transfer", "WAVE_SENEGAL"): r"D:\RECONCILIATIONS\TRANSFERT\WAVE - SN\Données PMT\NOV",
    ("Payment", "ORANGE_CAMEROUN"): r"D:\RECONCILIATIONS\PAIEMENT\ORANGE CAMEROUN\DONNEES PMT\2025\NOV",
    ("Transfer", "ORANGE_CAMEROUN"): r"D:\RECONCILIATIONS\TRANSFERT\OM CM\Données PMT\2025\NOV",
    ("Payment", "MOOV_CI"): r"D:\RECONCILIATIONS\PAIEMENT\MOOV CI\DONNEES PMT\NOV",
    ("Transfer", "MOOV_CI"): r"D:\RECONCILIATIONS\TRANSFERT\MOOV CI\DONNEES PMT\NOV",
}

# Étape 1 : Extraction du zip
def extract_zip(zip_path, extract_dir):
    if os.path.exists(extract_dir):
        shutil.rmtree(extract_dir)
    os.makedirs(extract_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

# Étape 2 : Copier avec renommage si fichier déjà présent
def safe_copy(src, dst):
    """Copie un fichier, en renommant s'il existe déjà"""
    if not os.path.exists(dst):
        shutil.copy2(src, dst)
        return dst

    base, ext = os.path.splitext(dst)
    i = 251110
    new_dst = f"{base}_{i}{ext}"
    while os.path.exists(new_dst):
        i += 1
        new_dst = f"{base}_{i}{ext}"
    shutil.copy2(src, new_dst)
    return new_dst

# Étape 3 : Dispatch avec mapping
def dispatch_files_with_mapping(extract_dir, mapping):
    for root, dirs, files in os.walk(extract_dir):
        for file in files:
            if file.endswith(".csv"):
                parts = os.path.relpath(root, extract_dir).split(os.sep)
                if len(parts) >= 2:
                    category = parts[0]
                    operator = parts[1]

                    key = (category, operator)
                    if key in mapping:
                        target_dir = mapping[key]
                        if os.path.exists(target_dir):
                            src = os.path.join(root, file)
                            dst = os.path.join(target_dir, file)
                            final_path = safe_copy(src, dst)
                            print(f"✅ {file} copié dans {final_path}")
                        else:
                            print(f"⚠️ Le dossier cible {target_dir} n’existe pas encore.")
                    else:
                        print(f"❌ Pas de mapping défini pour {category}/{operator}")

# Pipeline complet
extract_zip(zip_path, extract_dir)
dispatch_files_with_mapping(extract_dir, MAPPING)

❌ Pas de mapping défini pour Payment/AIRTEL_NIGER
✅ AIRTEL_RDC_extractions.csv copié dans D:\RECONCILIATIONS\PAIEMENT\AIRTEL CD\DONNESS PMT\NOV\AIRTEL_RDC_extractions_251110.csv
✅ AIRTEL_TCHAD_extractions.csv copié dans D:\RECONCILIATIONS\PAIEMENT\AIRTEL TD\DONNEES PMT\AIRTEL_TCHAD_extractions_251110.csv
❌ Pas de mapping défini pour Payment/BIZAO
✅ CINETPAY_extractions.csv copié dans D:\RECONCILIATIONS\PAIEMENT\Cinetpay\DONNEES PMT\2025\NOV\CINETPAY_extractions_251110.csv
❌ Pas de mapping défini pour Payment/FEDAPAY
❌ Pas de mapping défini pour Payment/FREE_SENEGAL
❌ Pas de mapping défini pour Payment/GIMAC
❌ Pas de mapping défini pour Payment/HUB2
✅ IFUTUR_extractions.csv copié dans D:\RECONCILIATIONS\PAIEMENT\IFUTUR\DONNEES PMT\NOV\IFUTUR_extractions_251110.csv
❌ Pas de mapping défini pour Payment/LIGDICASH
❌ Pas de mapping défini pour Payment/MAGMA
✅ MOOVBF_extractions.csv copié dans D:\RECONCILIATIONS\PAIEMENT\MOOVBF\DATA PMT\NOV\MOOVBF_extractions_251110.csv
✅ MOOV_CI_extractions.